## 215 - Creating PDF Reports in Python Part 2

[Youtube](https://www.youtube.com/watch?v=QLmnk22AQj0)

In [1]:
from fpdf import FPDF

In [2]:
class PDF(FPDF):
    def header(self):
        self.image('unidata.png', w = 20)
        self.set_font('Arial', 'B', 11)
        self.set_xy(0, 0)
        self.cell(200, 40, 'Current Buoy Summary', 0, 0, 'R')
        self.ln(20)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

In [3]:
from siphon.simplewebservice.ndbc import NDBC
from datetime import datetime

In [4]:
df = NDBC.latest_observations()
df.columns

df.describe()

,latitude,longitude,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,3hr_pressure_tendency,air_temperature,water_temperature,dewpoint,visibility,water_level_above_mean
count,843.000000,843.000000,597.000000,632.000000,552.000000,188.000000,146.000000,76.000000,92.000000,588.000000,349.000000,651.000000,530.000000,249.000000,45.000000,34.000000
mean,38.576202,-90.203019,167.705193,4.165823,5.491123,0.964894,7.945205,6.281579,188.728261,1016.123980,-0.599140,22.368203,23.264717,16.600402,9.342222,0.646471
std,12.667206,45.905385,94.140443,2.547135,3.049890,0.804028,4.004792,1.630927,87.547300,4.318896,0.847257,6.136949,6.816442,5.592770,5.235955,0.331497
min,-22.000000,-177.703000,0.000000,0.000000,0.000000,0.000000,2.000000,2.400000,23.000000,991.300000,-3.000000,7.600000,2.900000,5.800000,5.900000,0.090000
25%,29.533500,-122.315000,100.000000,2.100000,3.600000,0.300000,4.000000,5.575000,130.500000,1012.800000,-1.200000,17.000000,18.400000,11.100000,5.900000,0.467500
50%,39.582000,-85.972000,150.000000,3.600000,5.100000,0.800000,8.500000,6.500000,186.000000,1016.650000,-0.900000,24.100000,24.050000,16.700000,8.700000,0.550000
75%,46.211500,-76.556000,240.000000,5.700000,7.000000,1.425000,11.000000,7.225000,240.750000,1019.300000,0.000000,27.100000,29.400000,21.000000,11.000000,0.815000
max,70.400000,179.784000,360.000000,21.600000,25.700000,5.300000,15.000000,10.000000,352.000000,1023.800000,2.600000,33.900000,34.100000,28.300000,27.000000,1.530000


In [5]:
pdf = PDF()
pdf.add_page()
pdf.alias_nb_pages()
pdf.set_font('Arial', 'B', 16)
pdf.cell(60, 40, f'{datetime.utcnow():%Y-%m-%d %H:%M}')

report_vars = ['wind_speed', 'wind_gust', 'wave_height', 'dominant_wave_period', 'average_wave_period', 
               'dominant_wave_direction', 'pressure', 'air_temperature', 'water_temperature', 'dewpoint', 
               'visibility', 'water_level_above_mean']

df_describe = df.describe()
pdf.set_font('Arial', 'B', 12)
y_height = 5
pdf.set_xy(10, 50)
pdf.cell(25)
pdf.cell(60, y_height, 'Variable', border = 1)
pdf.cell(20, y_height, 'Mean', border = 1)
pdf.cell(20, y_height, 'Min', border = 1)
pdf.cell(20, y_height, 'Max', border = 1)
pdf.cell(20, y_height, 'Std Dev.', border = 1, ln = 1)

pdf.set_font('Arial', '', 12)
for var in report_vars:
    pdf.cell(25)
    pdf.cell(60, y_height, var, border = 1)
    pdf.cell(20, y_height, str(round(df_describe[var]['mean'], 1)), border = 1)
    pdf.cell(20, y_height, str(round(df_describe[var]['min'], 1)), border = 1)
    pdf.cell(20, y_height, str(round(df_describe[var]['max'], 1)), border = 1)
    pdf.cell(20, y_height, str(round(df_describe[var]['std'], 1)), border = 1, ln = 1)


pdf.output('report.pdf', 'F')

''